In [1]:
!pip install langchain faiss-cpu sentence-transformers transformers langchain-community

   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 12.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/13.7 MB ? eta -:--:--
   ---------------------- ----------------- 7.6/13.7 MB 39.0 MB/s eta 0:00:01
   -------------------------------- ------- 11.3/13.7 MB 33.5 MB/s eta 0:00:01
   ------------------------------------ --- 12.3/13.7 MB 20.3 MB/s eta 0:00:01
   ---------------------------------------- 13.7/13.7 MB 18.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 2.5/2.5 MB 18.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
   ------------ --------------------------- 5.0/15.8 MB 25.1 MB/s eta 0:00:01
   ------------------- -------------------- 7.9/15.8 MB 18.7 MB/s eta 0:00:01
   ------------------------- -------------- 10.2/15.8 MB 16.8 MB/s eta 0:00:01
   ---------------


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.chains import RetrievalQA